<a href="https://colab.research.google.com/github/dgsotiropoulos/Interval-Neural-Networks-in-Pytorch/blob/main/FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#ΕΙΣΑΓΩΓΗ ΒΙΒΛΙΟΘΗΚΩΝ
########## FINAL PROJECT###############

from IPython import get_ipython
get_ipython().magic('reset -sf')

############ Libraries ###############
import pandas as pd
import tensorflow as tf
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import numpy as np
import statistics

In [6]:
#ΕΙΣΑΓΩΓΗ ΑΡΧΕΙΟΥ-ΔΙΑΜΕΡΙΣΗ-ΚΑΝΟΝΙΚΟΠΟΙΗΣΗ
df = pd.read_csv('ΔΕΔΟΜΕΝΑ ΤΥΧΑΙΟΥ ΠΕΙΡΑΜΑΤΟΣ.csv',header=None,sep=",")
print(df.shape)
# Split data into X and Y
X=df.iloc[:,0:3]
Y=df.iloc[:,3].values
from sklearn.preprocessing import  MinMaxScaler
sc= MinMaxScaler()
X= sc.fit_transform(X)
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


FileNotFoundError: ignored

In [ ]:
#@title


In [ ]:
# Create model
model_aprox = Sequential()
model_aprox.add(Dense(200, activation='relu', input_dim=3))
model_aprox.add(Dense(100, activation='relu'))
model_aprox.add(Dense(1))

model_aprox.compile(optimizer="Adam", loss="mse")

# Train model
model_aprox.fit(X_train, Y_train, epochs=800, batch_size=10)

# Evaluate model
loss_aprox = model_aprox.evaluate(X_test, Y_test)
y_pred_aprox = model_aprox.predict(X_train)
print('Test loss:', loss_aprox)


counter_aprox_lower=0
counter_aprox_upperr=0
list_of_abs_dif_y_aprox=[]
for i in range(0,len(y_pred_aprox)):
  dif_aprox=Y_train[i]-y_pred_aprox[i]
  if dif_aprox>0:
    counter_aprox_lower=counter_aprox_lower+1
  else:
    counter_aprox_upperr=counter_aprox_upperr+1
  abs_dif_aprox=abs(Y_train[i]-y_pred_aprox[i])
  list_of_abs_dif_y_aprox=list_of_abs_dif_y_aprox+[[abs_dif_aprox]]

print("y_aprox_lower:",counter_aprox_lower,"y_aprox_upper:",counter_aprox_upperr)
max_dif=max(list_of_abs_dif_y_aprox)
min_dif=min(list_of_abs_dif_y_aprox)
print("Μεγιστη διαφορα: ",max_dif[0][0],"Ελάχιστη διαφορα: ",min_dif[0][0])

In [ ]:
#ΔΗΜΙΟΥΡΓΙΑ ΤΩΝ 2 ΝΕΥΡΩΝΙΚΩΝ ΔΙΚΤΥΩΝ LOWER ΚΑΙ UPPER



#Ορισμός μεταβλητής μετατόπισης
metatopisi=max_dif[0][0]+0.3
print("Η ματατοπιση της error function είναι :", metatopisi)



######### LOWER NN #################

# Define custom loss function
def custom_loss_lower(y_true, y_pred):
  lower=K.square(y_pred-y_true+metatopisi)
  loss=K.mean(lower)
  return loss

# Create model
model_lower = Sequential()
model_lower.add(Dense(200, activation='relu', input_dim=3))
model_lower.add(Dense(100, activation='relu'))
model_lower.add(Dense(1))

model_lower.compile(optimizer="Adam", loss=custom_loss_lower)

# Train model
model_lower.fit(X_train, Y_train, epochs=800, batch_size=10)

# Evaluate model
loss_lower = model_lower.evaluate(X_test, Y_test)
y_pred_lower = model_lower.predict(X_test)
print('Test loss:', loss_lower)


######### UPPER NN #################

# Define custom loss function
def custom_loss_upper(y_true, y_pred):
  upper=K.square(y_pred-y_true-metatopisi)
  loss=K.mean(upper)
  return loss

# Create model
model_upper = Sequential()
model_upper.add(Dense(200, activation='relu', input_dim=3))
model_upper.add(Dense(100, activation='relu'))
model_upper.add(Dense(1))

model_upper.compile(optimizer="Adam", loss=custom_loss_upper)

# Train model
model_upper.fit(X_train, Y_train, epochs=800, batch_size=10)

# Evaluate model
loss_upper = model_upper.evaluate(X_test, Y_test)
y_pred_upper = model_upper.predict(X_test)
print('Test loss:', loss_upper)








In [ ]:
# ΕΜΦΑΝΙΣΗ ΑΠΟΤΕΛΕΣΜΑΤΩΝ
#ΠΡΑΓΜΑΤΙΚΗ ΤΙΜΗ ΚΑΙ ΤΟ ΔΙΑΣΤΗΜΑ ΠΡΟΒΛΕΨΗΣ

# ΑΠΟΤΕΛΕΣΜΑ ΕΠΙΤΥΧΩΝ ΑΠΟΤΕΛΕΣΜΑΤΩΝ ΓΙΑ ΚΑΤΩ ΟΡΙΟ
counter_lower=0
counter_false_lower=0
for i in range(0,len(y_pred_lower)):
  dif_lower=Y_test[i]-y_pred_lower[i]
  if dif_lower>0:
    counter_lower=counter_lower+1
  else:
    counter_false_lower=counter_false_lower+1
print("ΑΠΟΤΕΛΕΣΜΑ ΕΠΙΤΥΧΩΝ ΑΠΟΤΕΛΕΣΜΑΤΩΝ ΓΙΑ ΚΑΤΩ ΟΡΙΟ")    
print("y_pred_lower:",counter_lower,"False:",counter_false_lower)

# ΑΠΟΤΕΛΕΣΜΑ ΕΠΙΤΥΧΩΝ ΑΠΟΤΕΛΕΣΜΑΤΩΝ ΓΙΑ ΠΑΝΩ ΟΡΙΟ
counter_upper=0
counter_false_upper=0
for i in range(0,len(y_pred_upper)):
  dif_upper=Y_test[i]-y_pred_upper[i]
  if dif_upper<0:
    counter_upper=counter_upper+1
  else:
    counter_false_upper=counter_false_upper+1
print("\nΑΠΟΤΕΛΕΣΜΑ ΕΠΙΤΥΧΩΝ ΑΠΟΤΕΛΕΣΜΑΤΩΝ ΓΙΑ ΠΑΝΩ ΟΡΙΟ")   
print("y_pred_upper:",counter_upper,"False:",counter_false_upper,"\n")
#for i in range (0,len(pososto_diaforas_upper)):
# print(abs(pososto_diaforas_upper[i][0]))
epitixia=0
for i in range (0,len(Y_test)):
  
  if Y_test[i]>y_pred_lower[i] and Y_test[i]<y_pred_upper[i]:
    epitixia="ΕΠΙΤΥΧΙΑ"
  else:
    epitixia="@@AΠΟΤΥΧΙΑ@@"
  
  range_interval=(y_pred_upper[i][0]-y_pred_lower[i][0])/2
  mid_point=(y_pred_upper[i][0]+y_pred_lower[i][0])/2
  print(i,"   Πραγματικη τιμή :",Y_test[i],"       ","Διάστημα πρόβλεψης :","[",y_pred_lower[i][0],y_pred_upper[i][0],"]    ",epitixia,"\n","  ΜΕΣΟ ΔΙΑΣΤΗΜΑΤΟΣ : ",mid_point,"  ΑΚΤΙΝΑ ΔΙΑΣΤΗΜΑΤΟΣ",range_interval, "\n" )